# Read Data and Build a Dataframe
### This part is used to read and decode the JSON data to a dataframe.

In [1]:
# Run some setup code for this notebook.
from __future__ import print_function

import os
import json
from progressbar import ProgressBar
import pandas as pd

In [2]:
#The dir of the data
dir=os.listdir("./data")

# The features we need of the match
names_match=[ 'radiant_win',
 'duration',
 'pre_game_duration',
 'start_time',
 'match_id',
 'match_seq_num',
 'tower_status_radiant',
 'tower_status_dire',
 'barracks_status_radiant',
 'barracks_status_dire',
 'cluster',
 'first_blood_time',
 'lobby_type',
 'human_players',
 'leagueid',
 'positive_votes',
 'negative_votes',
 'game_mode',
 'flags',
 'engine',
 'radiant_score',
 'dire_score']

# The features we need of the heroes
names_heroes=[
    'hero0',
    'hero1',
    'hero2',
    'hero3',
    'hero4',
    'hero5',
    'hero6',
    'hero7',
    'hero8',
    'hero9'
]

#Initial the dataframe of match
df_match=pd.DataFrame(columns=names_match)
#Inital the dataframe of Heroes
df_heroes=pd.DataFrame(columns=names_heroes)

In [3]:
df_heroes

,hero0,hero1,hero2,hero3,hero4,hero5,hero6,hero7,hero8,hero9


### This part you will remove some wrong data from the data dictionary and then read them.

In [4]:
# remove wrong data
pbar = ProgressBar()
for i in pbar(range(0,len(dir))):
    fopen=open("./data/"+dir[i],'r')
    b=json.load(fopen)
    if(len(b['result'])<23 or len(b['result'])>24):
        fopen.close()
        os.remove("./data/"+dir[i])
    fopen.close()

100% |########################################################################|


In [5]:
#Read all match data

#The dir of the new data 
dir=os.listdir("./data")

pbar = ProgressBar()
for i in pbar(range(0,len(dir))):
    fopen=open("./data/"+dir[i],'r')
    b=json.load(fopen)
    x=b['result']
    x.pop('players')
    if('picks_bans' in x.keys()):
        x.pop('picks_bans')
        
    df_match=df_match.append(x,ignore_index=True)   
    
    
    fopen.close()

100% |########################################################################|


In [6]:
df_match.head(5)

,radiant_win,duration,pre_game_duration,start_time,match_id,match_seq_num,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire,...,lobby_type,human_players,leagueid,positive_votes,negative_votes,game_mode,flags,engine,radiant_score,dire_score
0,False,1117,60,1551481517,4479300000,3850380705,6,1974,3,63,...,0,10,0,0,0,23,0,1,14,28
1,True,1068,60,1551481518,4479300001,3850380406,2038,48,63,12,...,0,10,0,0,0,23,0,1,23,13
2,True,3729,90,1551481518,4479300002,3850420517,0,0,0,51,...,7,10,0,0,0,22,0,1,69,55
3,False,1095,60,1551481518,4479300003,3850380492,256,2047,48,63,...,0,10,0,0,0,23,0,1,5,36
4,False,2541,90,1551481518,4479300004,3850403408,0,1975,0,63,...,0,10,0,0,0,22,0,1,29,48


### Do some data cleaning. Including omitting NaN and removing robot played match. Add a column called win_score

In [7]:
#Init win_score
df_match['win_score']=0

pbar = ProgressBar()
for i in pbar(range(0,len(df_match))):
    if(df_match.iloc[i,0]==True):
        df_match.iloc[i,22]=df_match.iloc[i,20]-df_match.iloc[i,21]
    else:
        df_match.iloc[i,22]=df_match.iloc[i,21]-df_match.iloc[i,20]

100% |########################################################################|


In [8]:
#Omit NAN
df_match=df_match.dropna(axis=0,how='any')

In [9]:
df_match.head(5)

,radiant_win,duration,pre_game_duration,start_time,match_id,match_seq_num,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire,...,human_players,leagueid,positive_votes,negative_votes,game_mode,flags,engine,radiant_score,dire_score,win_score
0,False,1117,60,1551481517,4479300000,3850380705,6,1974,3,63,...,10,0,0,0,23,0,1,14,28,14.0
1,True,1068,60,1551481518,4479300001,3850380406,2038,48,63,12,...,10,0,0,0,23,0,1,23,13,10.0
2,True,3729,90,1551481518,4479300002,3850420517,0,0,0,51,...,10,0,0,0,22,0,1,69,55,14.0
3,False,1095,60,1551481518,4479300003,3850380492,256,2047,48,63,...,10,0,0,0,23,0,1,5,36,31.0
4,False,2541,90,1551481518,4479300004,3850403408,0,1975,0,63,...,10,0,0,0,22,0,1,29,48,19.0


### This part is used to read heroes data


In [10]:
pbar = ProgressBar()
for index,row in df_match.iterrows():
    fopen=open("./data/"+str(row['match_id'])+'.json','r')
    b=json.load(fopen)
    x={}
    for j in range(0,10):
        x['hero'+str(j)]=b['result']['players'][j]['hero_id']
        
    df_heroes=df_heroes.append(x,ignore_index=True)
    
    fopen.close()

In [11]:
df_heroes.head(5)

,hero0,hero1,hero2,hero3,hero4,hero5,hero6,hero7,hero8,hero9
0,18,48,109,84,32,120,28,1,112,15
1,35,14,55,1,114,44,104,71,48,26
2,55,11,44,9,64,2,42,95,47,27
3,109,14,58,41,114,7,32,99,82,5
4,11,14,63,28,13,62,44,40,104,47


### Contat df_match and df_heroes. Then save the dataframe to "out.csv"

In [12]:
df=pd.concat([df_match,df_heroes],axis=1)    

In [13]:
df.head(5)

,radiant_win,duration,pre_game_duration,start_time,match_id,match_seq_num,tower_status_radiant,tower_status_dire,barracks_status_radiant,barracks_status_dire,...,hero0,hero1,hero2,hero3,hero4,hero5,hero6,hero7,hero8,hero9
0,False,1117,60,1551481517,4479300000,3850380705,6,1974,3,63,...,18,48,109,84,32,120,28,1,112,15
1,True,1068,60,1551481518,4479300001,3850380406,2038,48,63,12,...,35,14,55,1,114,44,104,71,48,26
2,True,3729,90,1551481518,4479300002,3850420517,0,0,0,51,...,55,11,44,9,64,2,42,95,47,27
3,False,1095,60,1551481518,4479300003,3850380492,256,2047,48,63,...,109,14,58,41,114,7,32,99,82,5
4,False,2541,90,1551481518,4479300004,3850403408,0,1975,0,63,...,11,14,63,28,13,62,44,40,104,47


In [14]:
# Save the data
df_match.reset_index(drop=True)
df_match.to_csv('sample.csv',index=None)